# Clarity Model Evaluation (COT)
This notebook evaluates the Clarity model with Chain-of-Thought prompting on:
1. Custom evaluation CSV dataset
2. HuggingFace test dataset with metrics

Uses 8-bit quantization (bitsandbytes) for single GPU inference.

In [ ]:
%pip install transformers accelerate datasets scikit-learn bitsandbytes -q

In [ ]:
import pandas as pd
import torch
import re
import warnings
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, classification_report, confusion_matrix

# Suppress attention mask warning
warnings.filterwarnings("ignore", message=".*attention_mask.*")

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name} ({props.total_memory / 1024**3:.1f} GB)")

In [ ]:
# === CONFIGURATION ===
MODEL_NAME = "Frenzyknight/Clarity-llama-70b"  # Your fine-tuned model
DATA_PATH = "workspace/clarity_task_evaluation_dataset.csv"  # Custom eval data
OUTPUT_PATH = "clarity_eval_predictions_cot.csv"  # Output for custom eval
HF_OUTPUT_PATH = "hf_test_predictions_cot.csv"  # Output for HF test
MAX_NEW_TOKENS = 1024  # COT model outputs reasoning + label
TEMPERATURE = 0.1
MAX_INPUT_LENGTH = 3500
LIMIT = None  # Set to 5 for testing, None for full run

In [ ]:
# System prompt (COT)
SYSTEM_PROMPT = "You are an expert political discourse analyst. Analyze political interviews step by step and classify response clarity."

# User prompt template (COT - with "Think step by step")
USER_TEMPLATE = """Based on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.

### Classification Categories ###
1. Clear Reply - The information requested is explicitly stated (in the requested form)
2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer
3. Ambivalent - The information requested is given in an incomplete way (e.g., the answer is too general, partial, implicit, dodging, or deflection)

### Full Interview Question ###
{interview_question}

### Full Interview Answer ###
{interview_answer}

### Specific Question to Classify ###
{question}

Think step by step, then provide your classification."""


def extract_label(response: str) -> str:
    """Extract label from COT response (looks for 'LABEL: <classification>')."""
    
    # Look for explicit LABEL: pattern (case insensitive)
    label_match = re.search(r'LABEL:\s*(Clear Reply|Clear Non-Reply|Ambivalent)', response, re.IGNORECASE)
    if label_match:
        label = label_match.group(1).strip()
        # Normalize capitalization
        if "non-reply" in label.lower():
            return "Clear Non-Reply"
        elif "clear reply" in label.lower():
            return "Clear Reply"
        elif "ambivalent" in label.lower():
            return "Ambivalent"
    
    # Fallback: check last 100 chars for label mentions
    last_part = response[-100:].lower() if len(response) > 100 else response.lower()
    if "clear non-reply" in last_part or "non-reply" in last_part:
        return "Clear Non-Reply"
    elif "clear reply" in last_part:
        return "Clear Reply"
    elif "ambivalent" in last_part:
        return "Ambivalent"
    
    return "PARSE_ERROR"


def format_messages_csv(row):
    """Format a CSV row into conversation messages."""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(
            interview_question=str(row["interview_question"]),
            interview_answer=str(row["interview_answer"]),
            question=str(row["question"])
        )}
    ]

print("✅ Helper functions defined")

In [ ]:
print("=" * 60)
print(f"Loading model: {MODEL_NAME}")
print("Using 4-bit quantization for single GPU...")
print("=" * 60)

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model with 4-bit quantization
print("Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)

print("\n✅ Model loaded successfully (4-bit quantized)!")
print(f"Model device map: {model.hf_device_map if hasattr(model, 'hf_device_map') else 'auto'}")

# Part A: Custom CSV Evaluation

In [ ]:
print(f"📂 Loading data from: {DATA_PATH}")
eval_df = pd.read_csv(DATA_PATH)
print(f"   Loaded {len(eval_df)} examples")
print(f"   Columns: {eval_df.columns.tolist()}")

if LIMIT:
    print(f"   Limiting to first {LIMIT} examples")
    eval_df = eval_df.head(LIMIT)

In [ ]:
csv_results = []
total = len(eval_df)

print("\n" + "=" * 60)
print(f"Starting inference on {total} examples (Custom CSV)")
print("=" * 60 + "\n")

for idx, row in tqdm(eval_df.iterrows(), total=total, desc="Evaluating CSV"):
    messages = format_messages_csv(row)
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    
    input_len = inputs.shape[-1]
    
    if input_len > MAX_INPUT_LENGTH:
        tqdm.write(f"[{idx + 1}/{total}] SKIPPED (input too long: {input_len} tokens)")
        csv_results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": "SKIPPED",
            "raw_output": f"Input too long: {input_len} tokens"
        })
        continue
    
    try:
        inputs = inputs.to(model.device)
        attention_mask = torch.ones_like(inputs)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=TEMPERATURE > 0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        full_response = tokenizer.decode(
            outputs[0, input_len:],
            skip_special_tokens=True
        ).strip()
        
        prediction = extract_label(full_response)
        
        csv_results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": prediction,
            "raw_output": full_response,
        })
        
    except Exception as e:
        tqdm.write(f"[{idx + 1}/{total}] ERROR: {e}")
        csv_results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": "ERROR",
            "raw_output": str(e),
        })
    
    if (idx + 1) % 25 == 0:
        torch.cuda.empty_cache()

print("\n✅ CSV Inference complete!")

In [ ]:
csv_results_df = pd.DataFrame(csv_results)
csv_results_df.to_csv(OUTPUT_PATH, index=False)

parse_errors = sum(1 for r in csv_results if r['prediction'] == 'PARSE_ERROR')
skipped = sum(1 for r in csv_results if r['prediction'] == 'SKIPPED')
errors = sum(1 for r in csv_results if r['prediction'] == 'ERROR')
successful = len(csv_results) - parse_errors - skipped - errors

print("\n" + "=" * 60)
print("CSV EVALUATION COMPLETE")
print("=" * 60)

print(f"\n✅ Saved {len(csv_results)} predictions to {OUTPUT_PATH}")

print("\n📊 Summary:")
print(f"   Total:         {len(csv_results)}")
print(f"   Successful:    {successful}")
print(f"   Parse errors:  {parse_errors}")
print(f"   Skipped:       {skipped}")
print(f"   Errors:        {errors}")

print("\n📈 Prediction distribution:")
for label in ["Clear Reply", "Clear Non-Reply", "Ambivalent"]:
    count = sum(1 for r in csv_results if r['prediction'] == label)
    pct = count / len(csv_results) * 100 if csv_results else 0
    print(f"   {label}: {count} ({pct:.1f}%)")

print("\n🔍 Sample outputs (first 5):")
for i, r in enumerate(csv_results[:5]):
    raw = r.get('raw_output', 'N/A')[:40]
    print(f"   {i+1}. {r['prediction']} | Raw: '{raw}'")

# Part B: HuggingFace Test Dataset (with Metrics)

In [ ]:
# Load test dataset from HuggingFace
hf_test = load_dataset("Frenzyknight/clarity-dataset", split="test")
print(f"✅ Loaded {len(hf_test)} test examples from HuggingFace")
print(f"   Columns: {hf_test.column_names}")
print(f"   Sample conversation structure: {len(hf_test[0]['conversations'])} messages")

In [ ]:
hf_results = []
total = len(hf_test)

print("\n" + "=" * 60)
print(f"Running inference on HuggingFace test set ({total} examples)")
print("=" * 60 + "\n")

for idx, example in tqdm(enumerate(hf_test), total=total, desc="Evaluating HF"):
    # Get the conversation (system + user messages already formatted)
    messages = example["conversations"]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    
    input_len = inputs.shape[-1]
    
    if input_len > MAX_INPUT_LENGTH:
        tqdm.write(f"[{idx + 1}/{total}] SKIPPED (input too long: {input_len} tokens)")
        hf_results.append({"prediction": "SKIPPED", "raw_output": "Too long"})
        continue
    
    try:
        inputs = inputs.to(model.device)
        attention_mask = torch.ones_like(inputs)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=TEMPERATURE > 0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        full_response = tokenizer.decode(
            outputs[0, input_len:],
            skip_special_tokens=True
        ).strip()
        
        prediction = extract_label(full_response)
        hf_results.append({"prediction": prediction, "raw_output": full_response})
        
    except Exception as e:
        tqdm.write(f"[{idx + 1}/{total}] ERROR: {e}")
        hf_results.append({"prediction": "ERROR", "raw_output": str(e)})
    
    if (idx + 1) % 25 == 0:
        torch.cuda.empty_cache()

print("\n✅ HuggingFace Inference complete!")

In [ ]:
# Load ground truth from QEvasion dataset (has clarity_label)
qevasion_test = load_dataset("ailsntua/QEvasion", split="test")

# Create DataFrame with predictions and ground truth
hf_df = pd.DataFrame(hf_results)
hf_df["clarity_label"] = [row["clarity_label"] for row in qevasion_test]
hf_df["annotator1"] = [row["annotator1"] for row in qevasion_test]
hf_df["annotator2"] = [row["annotator2"] for row in qevasion_test]
hf_df["annotator3"] = [row["annotator3"] for row in qevasion_test]

# Save to CSV
hf_df.to_csv(HF_OUTPUT_PATH, index=False)
print(f"✅ Saved predictions with labels to {HF_OUTPUT_PATH}")
print(f"\nDataFrame shape: {hf_df.shape}")
hf_df.head()

In [ ]:
# Filter to valid labels only
y_true = hf_df["clarity_label"].str.strip()
y_pred = hf_df["prediction"].str.strip()

VALID = {"Clear Reply", "Clear Non-Reply", "Ambivalent"}
mask = y_true.isin(VALID) & y_pred.isin(VALID)

y_true_filtered = y_true[mask]
y_pred_filtered = y_pred[mask]

print(f"Valid predictions: {mask.sum()} / {len(mask)}")
print(f"Filtered out: {(~mask).sum()} (PARSE_ERROR, SKIPPED, ERROR)")

In [ ]:
# Accuracy
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)

# Precision / Recall / F1 (Macro)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true_filtered,
    y_pred_filtered,
    average="macro"
)

# Weighted F1
f1_weighted = f1_score(
    y_true_filtered,
    y_pred_filtered,
    average="weighted"
)

print("=" * 60)
print("EVALUATION METRICS (COT Model on HuggingFace Test Set)")
print("=" * 60)
print(f"\nAccuracy        : {accuracy:.4f}")
print(f"Precision (Mac) : {precision_macro:.4f}")
print(f"Recall (Mac)    : {recall_macro:.4f}")
print(f"F1 (Macro)      : {f1_macro:.4f}")
print(f"F1 (Weighted)   : {f1_weighted:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 60 + "\n")

print(classification_report(
    y_true_filtered,
    y_pred_filtered,
    digits=4
))

In [ ]:
print("\n📊 Prediction vs Ground Truth Distribution:")
print("\nGround Truth:")
for label in ["Clear Reply", "Clear Non-Reply", "Ambivalent"]:
    count = (y_true_filtered == label).sum()
    print(f"   {label}: {count}")

print("\nPredictions:")
for label in ["Clear Reply", "Clear Non-Reply", "Ambivalent"]:
    count = (y_pred_filtered == label).sum()
    print(f"   {label}: {count}")

In [ ]:
labels = ["Clear Reply", "Clear Non-Reply", "Ambivalent"]
cm = confusion_matrix(y_true_filtered, y_pred_filtered, labels=labels)

print("\n📈 Confusion Matrix:")
print(f"{'':20} {'Pred CR':>12} {'Pred CNR':>12} {'Pred Amb':>12}")
print("-" * 60)
for i, true_label in enumerate(labels):
    row = f"{true_label:20}"
    for j in range(3):
        row += f" {cm[i][j]:>12}"
    print(row)

# View Results DataFrames

In [ ]:
print("CSV Results:")
csv_results_df.head(10)

In [ ]:
print("HuggingFace Test Results:")
hf_df.head(10)